Trying to connect the user to our models

In [7]:
import pandas as pd

X_test = pd.read_csv("./datasets/merged/X_test_imputed_scaled.csv",  index_col=0)

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import joblib

data_new = "datasets/merged/X_test_imputed_scaled.csv"

clf = "saved-models/merged/optimal_voting_rf_svc.pkl"

feat_cols = [
    "period_d", "rp_re", "insol_eflux", "eq_temp", "teff_k", "logg_cgs", "rstar_rsun"
]

def scaleAndInput(X_new):
    imputer = SimpleImputer(strategy='mean')
    X_new = pd.DataFrame(imputer.fit_transform(X_new), columns=feat_cols)

    scaler = StandardScaler()
    X_new = pd.DataFrame(scaler.fit_transform(X_new), columns=feat_cols)

    return X_new


def checkYourData(data, model=clf):
    X_new = pd.read_csv(data,  index_col=0)
    # X_new = scaleAndInput(X_new)

    clf = joblib.load(model)

    y_proba = clf.predict_proba(X_new)[:, 1]

    y_pred = (y_proba >= 0.4).astype(int)

    output_df = pd.DataFrame({
        "id": X_new.index,
        "prediction": y_pred,
        "probability": y_proba
    })
    output_df.to_csv("predictions_with_prob.csv", index=False)

    return output_df

df = checkYourData(data_new)


df.head(10)



,id,prediction,probability
0,0,1,0.691667
1,1,1,0.568333
2,2,0,0.123334
3,3,1,0.651944
4,4,0,0.304352
5,5,1,0.634444
6,6,1,0.531665
7,7,1,0.575556
8,8,1,0.592778
9,9,1,0.640278
